In [1]:
import pandas as pd
import math
import operator
import os
import shutil
from matplotlib import pyplot as plt
 

In [2]:
df = pd.read_csv('Brands_Categories_Sold_Lhr_encoded.csv')
df.rename(columns={'dates':'Date'},inplace=True)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df.set_index('Date',inplace=True)

In [3]:
directory = 'Working Data\Stocks\Monthly_Stocks_Data'
main_categ = []
for filename in os.listdir(directory):
    temp = filename.split()
    categ_name = temp[0] + ' ' + temp[1].split('-')[0]
    main_categ.append(categ_name)
       

In [4]:
for column in df.columns:
    if column not in main_categ:
        df.drop(column, inplace=True, axis=1)

In [9]:
df.shape[0]

1706

In [8]:
df.resample('M').sum()

,category r179f,category I108K,category I242N,category N189r,category Q239q,category Y159X,category B9q,category C176l,category v90D,category R72O,category X107H,category c27J,category d230v,category o52A,category a228a,category d206l,category M3Y,category z239x,category u85c,category e136a
Date,,,,,,,,,,,,,,,,,,,,
2017-04-30,103,10,1,4,61,4,2,1,1,0,11,2,17,23,6,0,0,0,5,0
2017-05-31,115,3,0,1,59,3,3,9,2,1,14,1,24,10,2,0,1,0,7,0
2017-06-30,270,12,3,5,2,6,0,2,2,4,22,19,49,38,8,0,5,0,11,0
2017-07-31,200,16,2,2,2,1,2,2,2,3,24,12,44,31,8,0,0,0,9,1
2017-08-31,276,24,6,4,11,8,13,5,3,9,26,14,131,42,5,0,5,0,5,26
2017-09-30,217,19,0,3,3,3,5,11,2,8,20,8,61,23,8,0,2,0,8,0
2017-10-31,155,5,3,3,1,5,4,9,1,3,27,5,41,45,13,6,6,0,7,0
2017-11-30,209,15,0,9,20,6,9,13,2,6,56,4,79,40,11,5,0,0,16,1
2017-12-31,131,12,1,4,29,8,4,11,4,1,40,2,59,39,19,5,4,0,2,2


In [ ]:
df.to_csv('Main_Categories.csv')

In [ ]:
df_cat_a228 = pd.read_csv('Stock Category Model Results\df_test_bilstm 500 epochs.csv')
df_cat_a228.drop('Unnamed: 0', inplace=True, axis=1)
df_cat_a228_lstm = pd.read_csv('Stock Category Model Results\df_test_lstm 500 epochs.csv')
df_cat_a228_lstm.drop('Unnamed: 0', inplace=True, axis=1)



In [ ]:
# for column in df_cat_a228.columns:
#     if "test" in column:
#         column_name_test = column
#         column_name_actual = column.replace(" test", "") + " actual"

#         c_test_data = df_cat_a228[column_name_test].values
#         c_actual_data = df_cat_a228[column_name_actual].values

#         plt.figure(figsize=(10,6))
#         title = column + ' bilstm'
#         plt.title(title)
#         plt.legend()
#         plt.plot(c_test_data)
#         plt.plot(c_actual_data)
#         plt.show()
#     else:
#         break



In [ ]:
# for column in df_cat_a228_lstm.columns:
#     if "test" in column:
#         column_name_test = column
#         column_name_actual = column.replace(" test", "") + " actual"

#         c_test_data = df_cat_a228_lstm[column_name_test].values
#         c_actual_data = df_cat_a228_lstm[column_name_actual].values

#         plt.figure(figsize=(10,6))
#         title = column + ' lstm'
#         plt.title(title)
#         plt.legend()
#         plt.plot(c_test_data)
#         plt.plot(c_actual_data)
#         plt.show()
#     else:
#         break

